In [1]:
import pdb2pqr
import pdb4amber
import numpy as np
import os
from pathlib import Path
import glob

In [2]:
# Nombre de la proteina de interes
prot_name = 'fxa'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P00742"

In [3]:
# Lista de archivos pdb para preparar, originalmente modelos generados con Modeller
pdb_list = glob.glob(F"../../FILES/CRYSTALS/PROT_{prot_name.upper()}_MODELOS_modeller_NoPrep/*")
pdb_list.sort()

In [4]:
main_out_dir = F"../../FILES/CRYSTALS/PROT_{prot_name.upper()}_PREPARED/"
out_dir_ens = main_out_dir +  "PREP_ALL_ENSAMBLE/" # Hidrógenos agregados sin pka
out_dir_PREP_PROT = main_out_dir + "PREP_PH_7/" # Hidrógenos agregados usando Propka (res ionizables a pH 7)
out_dir_pqr = main_out_dir + "PQR_PKAs/" # Directorio de pqr"
# Creación de los directorios, si estos no exiten
import os
if not os.path.exists(main_out_dir):
    os.makedirs(main_out_dir)
    os.makedirs(out_dir_ens)
    os.makedirs(out_dir_PREP_PROT)
    os.makedirs(out_dir_pqr)

In [5]:
?pdb4amber.run

Signature:
pdb4amber.run(
    arg_pdbout,
    arg_pdbin,
    arg_nohyd=False,
    arg_dry=False,
    arg_prot=False,
    arg_amber_compatible_residues=False,
    arg_strip_atom_mask=None,
    arg_mutate_string=None,
    arg_constph=False,
    arg_mostpop=False,
    arg_reduce=False,
    arg_no_reduce_db=False,
    arg_model=0,
    arg_add_missing_atoms=False,
    arg_elbow=False,
    arg_logfile='pdb4amber.log',
    arg_keep_altlocs=False,
    arg_leap_template=False,
    arg_conect=True,
    arg_noter=False,
)
Docstring: <no docstring>
File:      ~/miniconda3/envs/amber/lib/python3.7/site-packages/pdb4amber/pdb4amber.py
Type:      function


In [12]:
# Sufijos para nombrar los archivos resultantes
ens_f_tail = "_ENS"
prep_f_tail = "_PROT"
prep_anyways = False # True para volver a preparar TODAS las proteínas

for pdb_file in np.array(pdb_list):
    pdb_id = Path(pdb_file).stem.rsplit('_', 1)[0]
    print(F"Preparando la proteína: {pdb_id}.")
    # Omite el proceso si la proteína ya ha sido preparada previamente
    if prep_anyways:
        if os.path.isfile(out_dir_PREP_PROT  + pdb_id + prep_f_tail + ".pdb") \
            and os.path.isfile(out_dir_ens + pdb_id + ens_f_tail + ".pdb"):
            print(F"La proteína {pdb_id} ya existe en los directorios\n")
            continue
    
    # ENSAMBLE: Añade átomos faltantes, incluyendo Hs
    # No toma en cuenta los estados ionizables
    ensamble_file = out_dir_ens + pdb_id + ens_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin = pdb_file,
                  #arg_reduce=True, #Dejaré las conformaciones de ensamble sin hidrógenos por ahora
                  # UPDATE Agosto/2020:
                  #**********************
                  # La proteína 1c5m aparecía como completa, sin embargo ocacionó problemas al 
                  # momento de construir el ensamble, así que se corrigió todo de la siguiente manera
                  # 1) como se está trabajando con un fragmento de la proteína, se agregó desde 
                  # MODELLER (notebook 3) un residuo extra al final (S), para que TODAS las 
                  # conformaciones aparecieran incompletas y fueran forzadas a ser modeladas 
                  # (evitando saltarse 1c5m, a la cual le faltaba un átomo en la región C-terminal
                  # 2) Finalmente, se omite, para el ensamble, el comando reduce y en su lugar se 
                  # utiliza el comando add_missing_atoms que agrega hidrógenos sin estimar el pka de
                  #  los residuos, garantizando así el mismo número de átomos en todas las 
                  # conformaciones
                  #**********************
                  arg_add_missing_atoms = True,
                  ### Agosto 2020
                  arg_conect=False,
                  arg_constph=False,
                  arg_pdbout = ensamble_file)
    os.system("rm " + out_dir_ens + "/*" + ens_f_tail + "_*")

    
    # PROT PREPARADA:
    # Ejecuta pdb2pqr para obtener los estados de protonación de los residuos ionizables
    pqr_file = out_dir_pqr + pdb_id + "_PQR.pdb"
    os.system("pdb2pqr_cli --ff=amber --with-ph=7.0 --ffout=amber --ph-calc-method=propka --chain "
               + pdb_file + " " 
               + pqr_file )
    # Vuelve a ejecutar pdb4amber para generar las PROT con los estados ionizables usando "--reduce"
    prepared_file = out_dir_PREP_PROT  + pdb_id + prep_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin  = pdb_file,
                  arg_reduce = True,
                  arg_pdbout = prepared_file )
    # Elimina los archivos temporales de pdb4amber
    
    os.system("rm " + out_dir_PREP_PROT + "/*" + prep_f_tail + "_*")

Preparando la proteína: 1c5m.
Preparando la proteína: 1ezq.
Preparando la proteína: 1f0r.
Preparando la proteína: 1f0s.
Preparando la proteína: 1fax.
Preparando la proteína: 1fjs.
Preparando la proteína: 1g2l.
Preparando la proteína: 1g2m.
Preparando la proteína: 1hcg.
Preparando la proteína: 1ioe.
Preparando la proteína: 1iqe.
Preparando la proteína: 1iqf.
Preparando la proteína: 1iqg.
Preparando la proteína: 1iqh.
Preparando la proteína: 1iqi.
Preparando la proteína: 1iqj.
Preparando la proteína: 1iqk.
Preparando la proteína: 1iql.
Preparando la proteína: 1iqm.
Preparando la proteína: 1iqn.
Preparando la proteína: 1ksn.
Preparando la proteína: 1lpg.
Preparando la proteína: 1lpk.
Preparando la proteína: 1lpz.
Preparando la proteína: 1lqd.
Preparando la proteína: 1mq5.
Preparando la proteína: 1mq6.
Preparando la proteína: 1nfu.
Preparando la proteína: 1nfw.
Preparando la proteína: 1nfx.
Preparando la proteína: 1nfy.
Preparando la proteína: 1p0s.
Preparando la proteína: 1v3x.
Preparando